## Data Cleaning

#### Import modules

In [1]:
import os
import pandas as pd

#### Define paths and load data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
PATH = 'raw_data'

In [4]:
OUTPUT_PATH = 'outputs'

### Election results

#### Load data

In [5]:
results = pd.read_csv(os.path.join(PATH, 'HarvardData_countypres_2000-2020.csv'))
results = results[results['party'].isin(['DEMOCRAT', 'REPUBLICAN'])]
grpby_cols = ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
              'candidate', 'party', 'totalvotes', 'version']
results = results.groupby(grpby_cols, as_index=False).sum()

In [6]:
results['pctvotes'] = results['candidatevotes'] / results['totalvotes']
results['office'] = 'PRESIDENT'  # for consistency

#### Pivot table

In [7]:
index_cols = ['year', 'state', 'state_po', 'county_name', 
              'county_fips', 'office']

In [8]:
pivoted_results = pd.pivot_table(results, index=index_cols, values=['pctvotes', 'candidatevotes'], columns=['party']).reset_index()
pivoted_results.columns.name = ''
pivoted_results.columns = [(x[0] + '_' + x[1]).lower() if x[1] != '' else x[0] for x in  pivoted_results.columns.values]
a = pivoted_results['candidatevotes_democrat'] > pivoted_results['candidatevotes_republican']
pivoted_results['winner'] = ['democrat' if x else 'republican' for x in a]
pivoted_results = pivoted_results[['year', 'state', 'county_name', 'county_fips', 'candidatevotes_democrat',
                                   'candidatevotes_republican', 'pctvotes_democrat', 'pctvotes_republican', 'winner']]

In [9]:
pivoted_results.head()

,year,state,county_name,county_fips,candidatevotes_democrat,candidatevotes_republican,pctvotes_democrat,pctvotes_republican,winner
0,2000,ALABAMA,AUTAUGA,1001.0,4942.0,11993.0,0.287192,0.696943,republican
1,2000,ALABAMA,BALDWIN,1003.0,13997.0,40872.0,0.247822,0.723654,republican
2,2000,ALABAMA,BARBOUR,1005.0,5188.0,5096.0,0.499086,0.490236,democrat
3,2000,ALABAMA,BIBB,1007.0,2710.0,4273.0,0.381636,0.601746,republican
4,2000,ALABAMA,BLOUNT,1009.0,4977.0,12667.0,0.276915,0.704779,republican


Export election results

In [10]:
for year in pivoted_results['year'].unique():
    
    df_temp = pivoted_results[pivoted_results['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'results_{year}.csv'), index=False)

### Unemployment data

In [11]:
unemployment = pd.read_excel(os.path.join(PATH, 'ERS_Unemployment.xlsx'), header=4)

In [12]:
rates_cols = ['Unemployment_rate_2000',
              'Unemployment_rate_2004',
              'Unemployment_rate_2008',
              'Unemployment_rate_2012',
              'Unemployment_rate_2016',
              'Unemployment_rate_2020']

In [13]:
area_cols = ['State', 'Area_name', 'FIPS_Code']

In [14]:
unemployment_ey = unemployment[area_cols + rates_cols]

In [15]:
unemployment_ey.sample(2)

,State,Area_name,FIPS_Code,Unemployment_rate_2000,Unemployment_rate_2004,Unemployment_rate_2008,Unemployment_rate_2012,Unemployment_rate_2016,Unemployment_rate_2020
829,IA,"Clayton County, IA",19043,3.5,6.9,5.4,5.7,4.2,5.7
1620,MO,"Warren County, MO",29219,3.3,5.6,7.5,7.7,4.2,5.8


In [16]:
unemployment_ey_melted = pd.melt(unemployment_ey, 
                                 id_vars=area_cols, 
                                 value_vars=rates_cols, 
                                 value_name='unemployment',
                                 var_name='year')
unemployment_ey_melted['year'] = [x.split('_')[2] for x in unemployment_ey_melted['year']]
unemployment_ey_melted['year'] = unemployment_ey_melted['year'].astype(int)
unemployment_ey_melted.rename(columns={'FIPS_Code': 'county_fips'}, inplace=True)
unemployment_ey_melted = unemployment_ey_melted[['county_fips', 'year', 'unemployment']]

In [17]:
unemployment_ey_melted.sample()

,county_fips,year,unemployment
2100,39037,2000,4.3


Export unemployment data

In [18]:
for year in unemployment_ey_melted['year'].unique():
    
    df_temp = unemployment_ey_melted[unemployment_ey_melted['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'unemployment_{year}.csv'), index=False)

#### Population

In [19]:
population = pd.read_excel(os.path.join(PATH, 'ERS_PopulationEstimates.xls'), header=2)

In [20]:
population.sample()

,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017,POP_ESTIMATE_2018,POP_ESTIMATE_2019,N_POP_CHG_2010,N_POP_CHG_2011,N_POP_CHG_2012,N_POP_CHG_2013,N_POP_CHG_2014,N_POP_CHG_2015,N_POP_CHG_2016,N_POP_CHG_2017,N_POP_CHG_2018,N_POP_CHG_2019,Births_2010,Births_2011,Births_2012,Births_2013,Births_2014,Births_2015,Births_2016,Births_2017,Births_2018,Births_2019,Deaths_2010,Deaths_2011,Deaths_2012,Deaths_2013,Deaths_2014,Deaths_2015,Deaths_2016,Deaths_2017,Deaths_2018,Deaths_2019,NATURAL_INC_2010,NATURAL_INC_2011,NATURAL_INC_2012,NATURAL_INC_2013,NATURAL_INC_2014,NATURAL_INC_2015,NATURAL_INC_2016,NATURAL_INC_2017,NATURAL_INC_2018,NATURAL_INC_2019,INTERNATIONAL_MIG_2010,INTERNATIONAL_MIG_2011,INTERNATIONAL_MIG_2012,INTERNATIONAL_MIG_2013,INTERNATIONAL_MIG_2014,INTERNATIONAL_MIG_2015,INTERNATIONAL_MIG_2016,INTERNATIONAL_MIG_2017,INTERNATIONAL_MIG_2018,INTERNATIONAL_MIG_2019,DOMESTIC_MIG_2010,DOMESTIC_MIG_2011,DOMESTIC_MIG_2012,DOMESTIC_MIG_2013,DOMESTIC_MIG_2014,DOMESTIC_MIG_2015,DOMESTIC_MIG_2016,DOMESTIC_MIG_2017,DOMESTIC_MIG_2018,DOMESTIC_MIG_2019,NET_MIG_2010,NET_MIG_2011,NET_MIG_2012,NET_MIG_2013,NET_MIG_2014,NET_MIG_2015,NET_MIG_2016,NET_MIG_2017,NET_MIG_2018,NET_MIG_2019,RESIDUAL_2010,RESIDUAL_2011,RESIDUAL_2012,RESIDUAL_2013,RESIDUAL_2014,RESIDUAL_2015,RESIDUAL_2016,RESIDUAL_2017,RESIDUAL_2018,RESIDUAL_2019,GQ_ESTIMATES_BASE_2010,GQ_ESTIMATES_2010,GQ_ESTIMATES_2011,GQ_ESTIMATES_2012,GQ_ESTIMATES_2013,GQ_ESTIMATES_2014,GQ_ESTIMATES_2015,GQ_ESTIMATES_2016,GQ_ESTIMATES_2017,GQ_ESTIMATES_2018,GQ_ESTIMATES_2019,R_birth_2011,R_birth_2012,R_birth_2013,R_birth_2014,R_birth_2015,R_birth_2016,R_birth_2017,R_birth_2018,R_birth_2019,R_death_2011,R_death_2012,R_death_2013,R_death_2014,R_death_2015,R_death_2016,R_death_2017,R_death_2018,R_death_2019,R_NATURAL_INC_2011,R_NATURAL_INC_2012,R_NATURAL_INC_2013,R_NATURAL_INC_2014,R_NATURAL_INC_2015,R_NATURAL_INC_2016,R_NATURAL_INC_2017,R_NATURAL_INC_2018,R_NATURAL_INC_2019,R_INTERNATIONAL_MIG_2011,R_INTERNATIONAL_MIG_2012,R_INTERNATIONAL_MIG_2013,R_INTERNATIONAL_MIG_2014,R_INTERNATIONAL_MIG_2015,R_INTERNATIONAL_MIG_2016,R_INTERNATIONAL_MIG_2017,R_INTERNATIONAL_MIG_2018,R_INTERNATIONAL_MIG_2019,R_DOMESTIC_MIG_2011,R_DOMESTIC_MIG_2012,R_DOMESTIC_MIG_2013,R_DOMESTIC_MIG_2014,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
1025,21027,KY,Breckinridge County,8.0,8.0,7.0,7.0,1.0,20059,20053,20043,20272,20074,20042,19883,19979,19982,20131,20309,20477,-10.0,229.0,-198.0,-32.0,-159.0,96.0,3.0,149.0,178.0,168.0,44.0,203.0,228.0,216.0,223.0,222.0,251.0,256.0,226.0,230.0,70.0,210.0,255.0,223.0,216.0,241.0,225.0,244.0,239.0,214.0,-26.0,-7.0,-27.0,-7.0,7.0,-19.0,26.0,12.0,-13.0,16.0,0.0,-2.0,-1.0,-3.0,-2.0,5.0,8.0,15.0,9.0,9.0,17.0,237.0,-173.0,-19.0,-165.0,112.0,-29.0,124.0,182.0,143.0,17.0,235.0,-174.0,-22.0,-167.0,117.0,-21.0,139.0,191.0,152.0,-1.0,1.0,3.0,-3.0,1.0,-2.0,-2.0,-2.0,0.0,0.0,301.0,301.0,301.0,301.0,301.0,301.0,300.0,300.0,299.0,299.0,299.0,10.070693,11.302236,10.768771,11.170946,11.138428,12.562248,12.763942,11.177052,11.27838,10.417959,12.640658,11.117759,10.820288,12.091716,11.260979,12.165632,11.81998,10.493797,-0.347265,-1.338423,-0.348988,0.350657,-0.953289,1.301269,0.59831,-0.642928,0.784583,-0.099219,-0.049571,-0.149566,-0.100188,0.250865,0.40039,0.747887,0.445104,0.441328,11.75741,-8.575819,-0.947253,-8.265498,5.619387,-1.451415,6.182534,9.000989,7.01221,11.658192,-8.62539,-1.096819,-8.365686,5.870252,-1.051025,6.930422,9.446093,7.453538


We obtained the rates for natural increase in population, international migration, domestic migration, and net migration for the years 2012, 2016, and we substituted the value of 2019 for 2020. 

In [21]:
years_population = ['2012', '2016', '2019']

In [22]:
rate_nat_inc_cols = ['R_NATURAL_INC_' + str(x) for x in years_population]
rate_int_mig_cols = ['R_INTERNATIONAL_MIG_' + str(x) for x in years_population]
rate_dom_mig_cols = ['R_DOMESTIC_MIG_' + str(x) for x in years_population]
rate_net_mig_cols = ['R_NET_MIG_' + str(x) for x in years_population]

In [23]:
population_dict = {'rate_natural_increase_population': rate_nat_inc_cols,
                   'rate_international_migration': rate_int_mig_cols,
                   'rate_domestic_migration': rate_dom_mig_cols,
                   'rate_net_migration': rate_net_mig_cols}

In [45]:
for key in population_dict.keys():
    
    cols = population_dict[key]
    temp2 = pd.melt(population[['FIPStxt'] + cols], 
                    id_vars=['FIPStxt'], 
                    value_vars=cols, 
                    value_name=key,
                    var_name='year')
    temp2['year'] = [x.split("_")[3] for x in temp2['year']]
    temp2['year'] = ['2020' if x == '2019' else x for x in temp2['year']]
    
    for year in temp2['year'].unique():
        df_temp = temp2[temp2['year'] == year]
        df_temp.rename(columns={'FIPStxt': 'county_fips'}, inplace=True)
        df_temp.drop(['year'], inplace=True, axis=1)
        df_temp.to_csv(os.path.join(OUTPUT_PATH, f'{key}_{year}.csv'), index=False)
        

/home/lnbalon/miniconda3/envs/dsenv/lib/python3.6/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/lnbalon/miniconda3/envs/dsenv/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
natural_inc_year = pd.melt(population[['FIPStxt'] + rate_nat_inc_cols], 
                           id_vars=['FIPStxt'], 
                           value_vars=rate_nat_inc_cols, 
                           value_name='natural_increase_in_pop',
                           var_name='year')

natural_inc_year['year'] = [x.split("_")[3] for x in natural_inc_year['year']]
natural_inc_year['year'] = ['2020' if x == '2019' else x for x in natural_inc_year['year']]

#### Facebook

In [26]:
facebook = pd.read_csv(os.path.join(PATH, 'Facebook_Cornell_ERS_SocialConnectednessIndex.csv'))
facebook.rename(columns={'FIPS': 'county_fips'}, inplace=True)
facebook = facebook[['county_fips', 'sh050m', 'sh100m', 'sh500m']]

In [27]:
facebook.to_csv(os.path.join(OUTPUT_PATH, f'facebook_2014.csv'), index=False)

#### Social Capital

In [42]:
social_capital = pd.read_csv(os.path.join(PATH, 'PennState_AgriScienceDept_Religion_and_Social_Capital.csv'))
social_capital = social_capital.iloc[:, :18]
social_capital.drop(['County_Name', 'pop2014'], inplace=True, axis=1)
social_capital.rename(columns={'FIPS': 'county_fips'}, inplace=True)

In [44]:
social_capital.to_csv(os.path.join(OUTPUT_PATH, f'social_capital.csv'), index=False)